# `query_df` — A Practical Tutorial

This notebook shows how to use the **generic** `query_df` utility from `rosamllib.utils` to filter arbitrary pandas DataFrames using:
- Exact matches, wildcards (`*`, `?`) with escaping
- Ranges (`gte`, `lte`, `gt`, `lt`, `eq`, `neq`)
- Regex / inverse regex
- IN / NOT IN (`in`, `nin`)
- Existence checks (`exists`, `missing`)
- Container-aware queries (e.g., lists inside cells)
- Dot-paths into nested dicts/lists inside a cell (e.g., `Meta.series.desc`, `Meta.angles[0]`)
- Approximate numeric equality (`approx`)
- Callable, per-cell predicates
- Custom operators via `register_op`



## 0) Setup

Import the function and (optionally) the registry helpers so you can add custom operators later.

In [1]:
import sys, pandas as pd
from rosamllib.utils import query_df, register_op, _OPS
print('Python:', sys.version)
print('pandas:', pd.__version__)
print('query_df available ops:', sorted(_OPS.keys()))

Python: 3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]
pandas: 2.2.3
query_df available ops: ['NotRegEx', 'RegEx', 'approx', 'callable', 'contains', 'eq', 'exists', 'gt', 'gte', 'in', 'in_last_days', 'lt', 'lte', 'missing', 'neq', 'nin', 'time_between']


## 1) Create a toy dataset
We'll create a small DataFrame that mixes strings, numbers, lists, and nested dicts/lists.

In [2]:
import numpy as np
from datetime import date

df = pd.DataFrame([
    {
        'PatientID': 'P001',
        'Modality': 'CT',
        'Age': 42,
        'Score': 1.000001,
        'StudyDate': '2023-01-01',
        'Tags': ['head', 'contrast'],
        'Meta': {'series': {'desc': 'Head CT w/ contrast'}, 'angles': [0.0, 90.0]},
    },
    {
        'PatientID': 'P002',
        'Modality': 'MR',
        'Age': 35,
        'Score': 0.999999,
        'StudyDate': '2023-02-15',
        'Tags': ['knee'],
        'Meta': {'series': {'desc': 'Knee MR T1'}, 'angles': [15.0]},
    },
    {
        'PatientID': 'PX03',
        'Modality': 'CT',
        'Age': 60,
        'Score': 1.25,
        'StudyDate': '2023-03-01',
        'Tags': [],
        'Meta': {'series': {'desc': 'Chest CT'}, 'angles': []},
    },
    {
        'PatientID': 'P004',
        'Modality': None,
        'Age': 29,
        'Score': 1.0,
        'StudyDate': '2023-04-20',
        'Tags': ['head', 'noncontrast'],
        'Meta': {'series': {'desc': 'Head CT plain'}, 'angles': [0.0]},
    },
])

df

,PatientID,Modality,Age,Score,StudyDate,Tags,Meta
0,P001,CT,42,1.000001,2023-01-01,"[head, contrast]","{'series': {'desc': 'Head CT w/ contrast'}, 'a..."
1,P002,MR,35,0.999999,2023-02-15,[knee],"{'series': {'desc': 'Knee MR T1'}, 'angles': [..."
2,PX03,CT,60,1.250000,2023-03-01,[],"{'series': {'desc': 'Chest CT'}, 'angles': []}"
3,P004,None,29,1.000000,2023-04-20,"[head, noncontrast]","{'series': {'desc': 'Head CT plain'}, 'angles'..."


## 2) Exact and Wildcard Matching
- Exact: `{'column': 'value'}`
- Wildcards: `*` (many), `?` (one). Escape literal `*`/`?` with `\*`/`\?`.

In [3]:
query_df(df, PatientID='P00*')[['PatientID']]

,PatientID
0,P001
1,P002
3,P004


## 3) Regex & Case-Insensitive Matching
- Use `{'RegEx': pattern}` or `{'NotRegEx': pattern}`.
- Set `case_insensitive=True` to fold string comparisons and regex matches.

In [4]:
query_df(df, case_insensitive=True, Modality={'RegEx': '^c(t|oncomputed tomography)?$'})[['PatientID','Modality']]

C:\Users\yabdulkadir\Desktop\open_source\Github\rosamllib\src\rosamllib\utils\query_df.py:263: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return s2.str.contains(pattern, na=False, regex=True, flags=flags)


,PatientID,Modality
0,P001,CT
2,PX03,CT


In [5]:
query_df(
    df,
    case_insensitive=True,
    Modality={'RegEx': r'^(?:ct|computed tomography)$'}
)[['PatientID','Modality']]

,PatientID,Modality
0,P001,CT
2,PX03,CT


## 4) Ranges / Comparison Operators
Supported: `gte`, `lte`, `gt`, `lt`, `eq`, `neq`.

In [6]:
query_df(df, Age={'gte': 40, 'lte': 60})[['PatientID','Age']]

,PatientID,Age
0,P001,42
2,PX03,60


## 5) IN / NOT IN, Exists / Missing

In [7]:
display(query_df(df, Modality={'in': ['CT','PT']})[['PatientID','Modality']])
display(query_df(df, Modality={'nin': ['CT','MR']})[['PatientID','Modality']])
display(query_df(df, Modality={'exists': True})[['PatientID','Modality']])
display(query_df(df, Modality={'missing': True})[['PatientID','Modality']])

,PatientID,Modality
0,P001,CT
2,PX03,CT


,PatientID,Modality
3,P004,None


,PatientID,Modality
0,P001,CT
1,P002,MR
2,PX03,CT


,PatientID,Modality
3,P004,None


## 6) Container-Aware Queries
For list-like cells (e.g., `Tags`), `contains` checks membership. A plain `eq` with a scalar also falls back to membership when the cell is a container.

In [8]:
display(query_df(df, Tags={'contains': 'head'})[['PatientID','Tags']])
display(query_df(df, Tags='contrast')[['PatientID','Tags']])  # scalar membership fallback

,PatientID,Tags
0,P001,"[head, contrast]"
3,P004,"[head, noncontrast]"


,PatientID,Tags
0,P001,"[head, contrast]"


## 7) Approximate Numeric Equality
Use `approx` with `value` plus `atol`/`rtol`.

In [9]:
query_df(df, Score={'approx': {'value': 1.0, 'atol': 1e-6, 'rtol': 1e-6}})[['PatientID','Score']]

,PatientID,Score
0,P001,1.000001
1,P002,0.999999
3,P004,1.000000


## 8) Callable Predicates
Pass a function that takes a single cell and returns `True`/`False`.

In [10]:
query_df(df, Age=lambda a: a is not None and a % 5 == 0)[['PatientID','Age']]

,PatientID,Age
1,P002,35
2,PX03,60


## 9) Dot-Paths into Nested Dicts / Lists in a Cell
You can reference nested content like `Meta.series.desc` or `Meta.angles[0]`. The dot-path walker supports sequence indices and `[*]` to iterate all list items.

In [11]:
display(query_df(df, **{'Meta.series.desc': {'RegEx': 'Head.*'}})[['PatientID','Meta']])
display(query_df(df, **{'Meta.angles[0]': {'gte': 0, 'lte': 1}})[['PatientID','Meta']])

,PatientID,Meta
0,P001,"{'series': {'desc': 'Head CT w/ contrast'}, 'a..."
3,P004,"{'series': {'desc': 'Head CT plain'}, 'angles'..."


,PatientID,Meta
0,P001,"{'series': {'desc': 'Head CT w/ contrast'}, 'a..."
3,P004,"{'series': {'desc': 'Head CT plain'}, 'angles'..."


## 10) OR Across Multiple Conditions for the Same Column
Pass a list of conditions — they’re OR’ed **within the same column**; AND’ed across different columns.

In [12]:
query_df(df, Modality=['CT', {'RegEx': '^M'}])[['PatientID','Modality']]

,PatientID,Modality
0,P001,CT
1,P002,MR
2,PX03,CT


## 11) Combine Multiple Filters (AND semantics across columns)

In [13]:
query_df(
    df,
    Modality='CT',
    Tags={'contains': 'head'},
    **{'Meta.series.desc': {'RegEx': '.*contrast.*'}},
)[['PatientID','Modality','Tags','Meta']]

,PatientID,Modality,Tags,Meta
0,P001,CT,"[head, contrast]","{'series': {'desc': 'Head CT w/ contrast'}, 'a..."


## 12) Custom Operators via `register_op`
You can add domain-specific operators without modifying `query_df`. An operator receives the column *Series*, the user-supplied value, and a context dict; it must return a boolean mask.

In [14]:
def op_near_int(series, target, ctx):
    """True if value is within 1 of target integer (handles NaN)."""
    try:
        tgt = int(target)
    except Exception:
        return pd.Series(False, index=series.index)
    return (series.astype('float64') - tgt).abs().le(1)

register_op('near_int', op_near_int)
print('Registered ops:', sorted(_OPS.keys()))
query_df(df, Age={'near_int': 35})[['PatientID','Age']]

Registered ops: ['NotRegEx', 'RegEx', 'approx', 'callable', 'contains', 'eq', 'exists', 'gt', 'gte', 'in', 'in_last_days', 'lt', 'lte', 'missing', 'near_int', 'neq', 'nin', 'time_between']


,PatientID,Age
1,P002,35


# 13) Dates & times (best practices)

In [15]:
df = pd.DataFrame({
    "PatientID": ["A","B","C","D"],
    "StudyDate": ["2024-02-15", "2024-03-01", "2023-12-31", "2024-03-15"],  # dates
    "AcqDateTime": [
        "2024-03-01T08:12:00Z",
        "2024-03-01T23:59:59Z",
        "2023-12-31T23:00:00Z",
        "2024-03-15T00:00:01Z",
    ],  # datetimes
    "SeriesTime": ["08:30:00","23:30:00","01:15:00","22:15:00"],  # time-of-day
})

# Coerce columns to appropriate dtypes once (recommended)
df["StudyDate"]   = pd.to_datetime(df["StudyDate"], errors="coerce").dt.date
df["AcqDateTime"] = pd.to_datetime(df["AcqDateTime"], errors="coerce", utc=True)
df["SeriesTime"]  = pd.to_datetime(df["SeriesTime"], errors="coerce", format="%H:%M:%S").dt.time

df

,PatientID,StudyDate,AcqDateTime,SeriesTime
0,A,2024-02-15,2024-03-01 08:12:00+00:00,08:30:00
1,B,2024-03-01,2024-03-01 23:59:59+00:00,23:30:00
2,C,2023-12-31,2023-12-31 23:00:00+00:00,01:15:00
3,D,2024-03-15,2024-03-15 00:00:01+00:00,22:15:00


In [16]:
# Range queries on date/datetime
# Acquisitions on/after March 1, 2024 (UTC)
query_df(df, AcqDateTime={"gte": "2024-03-01T00:00:00Z"})[
    ["PatientID","AcqDateTime"]
]

,PatientID,AcqDateTime
0,A,2024-03-01 08:12:00+00:00
1,B,2024-03-01 23:59:59+00:00
3,D,2024-03-15 00:00:01+00:00


In [17]:
query_df(df, StudyDate={"eq": "2024-02-15"})[["PatientID","StudyDate"]]

,PatientID,StudyDate
0,A,2024-02-15


In [18]:
# Time-of-day comparisons
# After 14:00:00
query_df(df, SeriesTime={"gte": "14:00:00"})[["PatientID","SeriesTime"]]

# Exactly 08:30:00
query_df(df, SeriesTime={"eq": "08:30:00"})[["PatientID","SeriesTime"]]

,PatientID,SeriesTime
0,A,08:30:00


In [19]:
# Usage (e.g., last 7 days)
query_df(df, AcqDateTime={"in_last_days": 7})[
    ["PatientID","AcqDateTime"]
]

,PatientID,AcqDateTime


## 14) Notes & Tips
- **Dot-paths** only traverse within a single cell (e.g., a `dict`/`list` stored in a column). They do not join across rows/tables.
- For **date**-like strings, prefer converting your DataFrame columns to `datetime64[ns]` first for efficient range queries.
- Use `case_insensitive=True` to normalize string comparisons and regex.
- For **float** comparisons, prefer `approx` over `eq` to avoid surprise misses due to precision.
- You can add any number of custom operators with `register_op(name, fn)`.

That’s it — happy querying!

In [20]:
import datetime as dt

In [21]:
df = pd.DataFrame({
    "PatientID": ["P1","P2","P3","P4"],
    "StudyDate": pd.to_datetime(["2024-02-14","2024-02-15","2024-02-20","2024-01-01"]),
    "SeriesTime": [dt.time(8,30), dt.time(14,5), dt.time(23,45), dt.time(3,15)],
    "SeriesDT": pd.to_datetime(["2024-02-14 08:30:00",
                                "2024-02-15 14:05:00",
                                "2024-02-20 23:45:00",
                                "2024-01-01 03:15:00"])
})

# 1) Time-of-day window 08:00..15:00 (inclusive)
query_df(df, SeriesTime={"time_between": ["08:00", "15:00"]})[["PatientID","SeriesTime"]]

# 2) Wrap-around window 22:00..06:00 (overnight)
query_df(df, SeriesTime={"time_between": ["22:00", "06:00"]})[["PatientID","SeriesTime"]]

# 3) Last 30 days by StudyDate (date column)
query_df(df, StudyDate={"in_last_days": 30})[["PatientID","StudyDate"]]

# 4) Last 7 days by SeriesDT (datetime column), with fixed reference "now"
ref_now = dt.datetime(2024, 2, 21, 12, 0, 0)
query_df(df, SeriesDT={"in_last_days": {"days": 7, "now": ref_now}})[["PatientID","SeriesDT"]]

,PatientID,SeriesDT
1,P2,2024-02-15 14:05:00
2,P3,2024-02-20 23:45:00
